<a href="https://colab.research.google.com/github/lucagiovannini7/emdracor/blob/main/scripts/%7BFre%2CEng%7DDraCor_metadata_enrichment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enriching metadata for FreDraCor and EngDraCor

In [ ]:
# imports
import pandas as pd
import numpy as np
import requests
import re
import json
import time
from tqdm import tqdm
!pip install urllib.requests
from urllib.request import urlopen
from statistics import mean
from sklearn import preprocessing

ERROR: Could not find a version that satisfies the requirement urllib.requests (from versions: none)
ERROR: No matching distribution found for urllib.requests


## 1. Load corpus metadata

In [ ]:
# load basic metadata csv from github

language = input("fre or eng, the choice is yours: ")
url = f"https://dracor.org/api/v1/corpora/{language}/metadata/csv"
metadata_table = pd.read_csv(url)
metadata_table

fre or eng, the choice is yours: eng


,name,id,wikidataId,firstAuthor,numOfCoAuthors,title,subtitle,normalizedGenre,digitalSource,originalSourcePublisher,...,numEdges,yearWritten,numOfSegments,wikipediaLinkCount,numOfActs,wordCountText,wordCountSp,wordCountStage,numOfP,numOfL
0,alexander-caesar,ep000318,NaN,Alexander,0,Caesar,NaN,Tragedy,https://texts.earlyprint.org/works/A16527_04.xml,NaN,...,21,1604,9,NaN,5,26908,21407,48,33,2584
1,alexander-croesus,ep000315,NaN,Alexander,0,Croesus,NaN,Tragedy,https://texts.earlyprint.org/works/A16527_01.xml,NaN,...,18,1604,9,NaN,5,24397,20156,28,10,2422
2,alexander-darius,ep000316,NaN,Alexander,0,Darius,NaN,Tragedy,https://texts.earlyprint.org/works/A16527_02.xml,NaN,...,25,1603,9,NaN,5,19153,14453,33,45,1683
3,alexander-the-alexandraean-tragedy,ep000317,NaN,Alexander,0,The Alexandraean Tragedy,NaN,Tragedy,https://texts.earlyprint.org/works/A16527_03.xml,NaN,...,35,1605,9,NaN,5,28756,23337,36,23,2817
4,anon-a-knack-to-know-a-knave,ep000117,Q49000520,Anon.,0,A Knack to Know a Knave,NaN,Comedy,https://texts.earlyprint.org/works/A04888.xml,NaN,...,406,1592,1,0.0,0,17196,16874,247,71,1654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,wilson-the-three-ladies-of-london,ep000311,Q7769016,Wilson,0,The Three Ladies of London,NaN,Comedy,https://texts.earlyprint.org/works/A15519.xml,NaN,...,253,1581,1,0.0,1,19791,18765,512,1,1633
430,wilson-the-three-lords-and-three-ladies-of-london,ep000310,NaN,Wilson,0,The Three Lords and Three Ladies of London,NaN,NaN,https://texts.earlyprint.org/works/A15517.xml,NaN,...,406,1588,2,NaN,0,22563,21390,1076,628,1121
431,woodes-the-conflict-of-conscience,ep000799,NaN,Woodes,0,The Conflict of Conscience,NaN,Comedy,https://texts.earlyprint.org/works/A68918.xml,NaN,...,36,1572,16,NaN,6,21364,19186,60,0,1946
432,yarington-two-lamentable-tragedies,ep000313,NaN,Yarington,0,Two Lamentable Tragedies,NaN,Tragedy,https://texts.earlyprint.org/works/A15814.xml,NaN,...,820,1594,1,NaN,0,22130,21421,765,8,2598


In [ ]:
#print(list(metadata_table))

In [ ]:
# drop unnecessary (= non numeric) columns

metadata_table = metadata_table.drop(columns=['wikidataId','firstAuthor', 'numOfCoAuthors', 'title', 'subtitle',
                                              'libretto', 'maxDegreeIds', 'yearPremiered', 'datePremiered', 'yearPrinted', 'yearWritten',
                                              'wikipediaLinkCount','originalSourceNumberOfPages','digitalSource',
                                              'originalSourcePublisher', 'originalSourcePubPlace','originalSourceYear'])
metadata_table = metadata_table.fillna(0)
metadata_table

,name,id,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,numOfPersonGroups,numConnectedComponents,numEdges,numOfSegments,numOfActs,wordCountText,wordCountSp,wordCountStage,numOfP,numOfL
0,alexander-caesar,ep000318,Tragedy,1604,11,0.478788,0.381818,1.600000,3.818182,3,...,1,2,21,9,5,26908,21407,48,33,2584
1,alexander-croesus,ep000315,Tragedy,1604,11,0.701948,0.327273,1.513514,3.272727,2,...,1,2,18,9,5,24397,20156,28,10,2422
2,alexander-darius,ep000316,Tragedy,1603,14,0.733333,0.274725,2.428571,3.571429,5,...,0,1,25,9,5,19153,14453,33,45,1683
3,alexander-the-alexandraean-tragedy,ep000317,Tragedy,1605,16,0.645238,0.291667,1.772152,4.375000,4,...,1,3,35,9,5,28756,23337,36,23,2817
4,anon-a-knack-to-know-a-knave,ep000117,Comedy,1592,29,1.000000,1.000000,1.000000,28.000000,1,...,0,1,406,1,0,17196,16874,247,71,1654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,wilson-the-three-ladies-of-london,ep000311,Comedy,1581,23,1.000000,1.000000,1.000000,22.000000,1,...,0,1,253,1,1,19791,18765,512,1,1633
430,wilson-the-three-lords-and-three-ladies-of-london,ep000310,0,1588,30,0.966667,0.933333,1.000000,27.066667,1,...,0,2,406,2,0,22563,21390,1076,628,1121
431,woodes-the-conflict-of-conscience,ep000799,Comedy,1572,17,0.695755,0.264706,1.733333,4.235294,3,...,0,3,36,16,6,21364,19186,60,0,1946
432,yarington-two-lamentable-tragedies,ep000313,Tragedy,1594,41,1.000000,1.000000,1.000000,40.000000,1,...,0,1,820,1,0,22130,21421,765,8,2598


In [ ]:
metadata_table.columns

Index(['name', 'id', 'normalizedGenre', 'yearNormalized', 'size',
       'averageClustering', 'density', 'averagePathLength', 'averageDegree',
       'diameter', 'maxDegree', 'numOfSpeakers', 'numOfSpeakersFemale',
       'numOfSpeakersMale', 'numOfSpeakersUnknown', 'numOfPersonGroups',
       'numConnectedComponents', 'numEdges', 'numOfSegments', 'numOfActs',
       'wordCountText', 'wordCountSp', 'wordCountStage', 'numOfP', 'numOfL'],
      dtype='object')

In [ ]:
# AFTER ENRICHMENT
# count how many without genre
#metadata_table.normalizedGenre.fillna('Other',inplace=True)  # still to fix, replace NaNs is not working
metadata_table.normalizedGenre.value_counts()
#df = metadata_table.loc[metadata_table['normalizedGenre'] == 0]
#df

# needs to further check English titles

normalizedGenre
Comedy         205
Tragedy        103
0               63
Tragicomedy     63
Name: count, dtype: int64

In [ ]:
metadata_table = metadata_table[(metadata_table['yearNormalized'] > 1561) & (metadata_table['yearNormalized'] < 1710)]
metadata_table


,name,id,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,numOfPersonGroups,numConnectedComponents,numEdges,numOfSegments,numOfActs,wordCountText,wordCountSp,wordCountStage,numOfP,numOfL
0,alexander-caesar,ep000318,Tragedy,1604,11,0.478788,0.381818,1.600000,3.818182,3,...,1,2,21,9,5,26908,21407,48,33,2584
1,alexander-croesus,ep000315,Tragedy,1604,11,0.701948,0.327273,1.513514,3.272727,2,...,1,2,18,9,5,24397,20156,28,10,2422
2,alexander-darius,ep000316,Tragedy,1603,14,0.733333,0.274725,2.428571,3.571429,5,...,0,1,25,9,5,19153,14453,33,45,1683
3,alexander-the-alexandraean-tragedy,ep000317,Tragedy,1605,16,0.645238,0.291667,1.772152,4.375000,4,...,1,3,35,9,5,28756,23337,36,23,2817
4,anon-a-knack-to-know-a-knave,ep000117,Comedy,1592,29,1.000000,1.000000,1.000000,28.000000,1,...,0,1,406,1,0,17196,16874,247,71,1654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,wilson-the-three-ladies-of-london,ep000311,Comedy,1581,23,1.000000,1.000000,1.000000,22.000000,1,...,0,1,253,1,1,19791,18765,512,1,1633
430,wilson-the-three-lords-and-three-ladies-of-london,ep000310,0,1588,30,0.966667,0.933333,1.000000,27.066667,1,...,0,2,406,2,0,22563,21390,1076,628,1121
431,woodes-the-conflict-of-conscience,ep000799,Comedy,1572,17,0.695755,0.264706,1.733333,4.235294,3,...,0,3,36,16,6,21364,19186,60,0,1946
432,yarington-two-lamentable-tragedies,ep000313,Tragedy,1594,41,1.000000,1.000000,1.000000,40.000000,1,...,0,1,820,1,0,22130,21421,765,8,2598


In [ ]:
# remove problematic play "anonyme-vende"
if language == "fre":
    metadata_table = metadata_table[metadata_table['name'] != 'anonyme-vende']
    metadata_table.shape

In [ ]:
# TEST -- get all type of useful metadata through the API
"""
metadata_url = "http://dracor.org/api/v1/corpora/fre/plays/racine-phedre"
response1 = urlopen(metadata_url)
metadata = json.loads(response1.read())

spoken_text_url = "http://dracor.org/api/v1/corpora/fre/plays/racine-phedre/spoken-text-by-character"
spoken_text = requests.get(spoken_text_url).json()

network_metrics_url = "http://dracor.org/api/v1/corpora/fre/plays/racine-phedre/metrics"
response2 = urlopen(network_metrics_url)
network_metrics = json.loads(response2.read())

characters_url = "http://dracor.org/api/v1/corpora/fre/plays/racine-phedre/characters"
response3 = requests.get(characters_url)
character_metrics = response3.json()
"""
# this will be wrapped in a function

'\nmetadata_url = "http://dracor.org/api/v1/corpora/fre/plays/racine-phedre"\nresponse1 = urlopen(metadata_url)\nmetadata = json.loads(response1.read())\n\nspoken_text_url = "http://dracor.org/api/v1/corpora/fre/plays/racine-phedre/spoken-text-by-character"\nspoken_text = requests.get(spoken_text_url).json()\n\nnetwork_metrics_url = "http://dracor.org/api/v1/corpora/fre/plays/racine-phedre/metrics"\nresponse2 = urlopen(network_metrics_url)\nnetwork_metrics = json.loads(response2.read())\n\ncharacters_url = "http://dracor.org/api/v1/corpora/fre/plays/racine-phedre/characters"\nresponse3 = requests.get(characters_url)\ncharacter_metrics = response3.json()\n'

## 2. Metrics enrichment


#### 0. Adding centrality measures via API
Betweenness centrality, closeness centrality, eigenvector centrality

In [ ]:
def get_centralities(network_metrics):

    from statistics import mean
    betweennesses = []
    closenesses = []
    eigenvectors = []
    for el in network_metrics['nodes']:
      betweennesses.append(el['betweenness'])
      closenesses.append(el['closeness'])
      eigenvectors.append(el['eigenvector'])
    avg_betweenness = mean(betweennesses)
    avg_closeness = mean(closenesses)
    avg_eigenvector = mean(eigenvectors)

    return avg_betweenness, avg_closeness, avg_eigenvector

In [ ]:
network_metrics_url = "http://dracor.org/api/v1/corpora/fre/plays/viau-pyrame/metrics"
response2 = urlopen(network_metrics_url)
network_metrics = json.loads(response2.read())
get_centralities(network_metrics)

(0.045454545454545456, 0.24440360775588046, 0.21300360530653817)

In [ ]:
# prepare empty columns

data = {'avg_betweenness': [],
        'avg_closeness': [],
        'avg_eigenvector': []}

extra_features = pd.DataFrame(data)
extra_features

,avg_betweenness,avg_closeness,avg_eigenvector


In [ ]:
for slug in tqdm(metadata_table['name'], desc='Processing plays'):
    url = f"http://dracor.org/api/corpora/{language}/play/{slug}/metrics"
    print("Computing centrality measures for", url)
    response = urlopen(url)
    network_metrics = json.loads(response.read())
    try:
        avg_betweenness, avg_closeness, avg_eigenvector = get_centralities(network_metrics)
    except Exception as e:
        print("This play throws an error:", url, e)
        continue

    new_row = {
        'avg_betweenness': avg_betweenness,
        'avg_closeness': avg_closeness,
        'avg_eigenvector': avg_eigenvector
    }
    extra_features = pd.concat([extra_features, pd.DataFrame([new_row])], ignore_index=True)

Processing plays:   0%|          | 0/429 [00:00<?, ?it/s]

Computing centrality measures for http://dracor.org/api/corpora/eng/play/alexander-caesar/metrics


Processing plays:   0%|          | 1/429 [00:00<01:48,  3.95it/s]

Computing centrality measures for http://dracor.org/api/corpora/eng/play/alexander-croesus/metrics


Processing plays:   0%|          | 2/429 [00:00<01:42,  4.17it/s]

...

Processing plays: 100%|█████████▉| 427/429 [01:31<00:00,  4.77it/s]

Computing centrality measures for http://dracor.org/api/corpora/eng/play/yarington-two-lamentable-tragedies/metrics


Processing plays: 100%|█████████▉| 428/429 [01:31<00:00,  4.75it/s]

Computing centrality measures for http://dracor.org/api/corpora/eng/play/zouche-the-sophister/metrics


Processing plays: 100%|██████████| 429/429 [01:31<00:00,  4.67it/s]


In [ ]:
extra_features.reset_index(drop=True, inplace=True)
metadata_table.reset_index(drop=True, inplace=True)

In [ ]:
# Concatenate the DataFrames horizontally
table1 = pd.concat([metadata_table, extra_features], axis=1)

In [ ]:
table1

,name,id,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,numOfSegments,numOfActs,wordCountText,wordCountSp,wordCountStage,numOfP,numOfL,avg_betweenness,avg_closeness,avg_eigenvector
0,alexander-caesar,ep000318,Tragedy,1604,11,0.478788,0.381818,1.600000,3.818182,3,...,9,5,26908,21407,48,33,2584,0.054545,0.524548,0.266490
1,alexander-croesus,ep000315,Tragedy,1604,11,0.701948,0.327273,1.513514,3.272727,2,...,9,5,24397,20156,28,10,2422,0.038384,0.459898,0.251954
2,alexander-darius,ep000316,Tragedy,1603,14,0.733333,0.274725,2.428571,3.571429,5,...,9,5,19153,14453,33,45,1683,0.119048,0.424969,0.229104
3,alexander-the-alexandraean-tragedy,ep000317,Tragedy,1605,16,0.645238,0.291667,1.772152,4.375000,4,...,9,5,28756,23337,36,23,2817,0.036310,0.390252,0.198640
4,anon-a-knack-to-know-a-knave,ep000117,Comedy,1592,29,1.000000,1.000000,1.000000,28.000000,1,...,1,0,17196,16874,247,71,1654,0.000000,1.000000,0.185695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,wilson-the-three-ladies-of-london,ep000311,Comedy,1581,23,1.000000,1.000000,1.000000,22.000000,1,...,1,1,19791,18765,512,1,1633,0.000000,1.000000,0.208514
425,wilson-the-three-lords-and-three-ladies-of-london,ep000310,0,1588,30,0.966667,0.933333,1.000000,27.066667,1,...,2,0,22563,21390,1076,628,1121,0.000000,0.933333,0.179506
426,woodes-the-conflict-of-conscience,ep000799,Comedy,1572,17,0.695755,0.264706,1.733333,4.235294,3,...,16,6,21364,19186,60,0,1946,0.037745,0.458304,0.207882
427,yarington-two-lamentable-tragedies,ep000313,Tragedy,1594,41,1.000000,1.000000,1.000000,40.000000,1,...,1,0,22130,21421,765,8,2598,0.000000,1.000000,0.156174


#### 1. Adding metrics from Trilcke et al. 2017

In [ ]:
#all-in index
def compute_all_in_index(metadata):
  all_in_index = metadata['allInIndex']
  return all_in_index

In [ ]:
#final scene size
#last_segment = metadata['segments'][-1]['number']
def compute_final_scene_size(metadata):
  final_scene_size = len(metadata['segments'][-1]['speakers'])/len(metadata['characters'])
  return final_scene_size

In [ ]:
''' [...] we introduced a measure we call the drama-change rate. The basis of our calculation is a modified Levenshtein distance, which only takes into account insertions ("add character") and deletions ("delete character").
In each case, we compare characters present in two consecutive scenes, eventually resulting in what we call the segment-change rate. The sum of all segment-change rates of a drama divided by the number of all
segment-change rates is what we call the drama-change rate '''

# segment change rate = no. of edits / total no. of characters

def compute_change_rate(metadata):

  num_edits = 0
  num_characters = 0
  segment_change_rate = []
  scene1 = 0
  scene2 = 1

  try:
      while scene2 < len(metadata['segments']):

        # 1. COUNTING DELETED SPEAKERS
        if 'speakers' in metadata['segments'][scene1]: # if speakers exist in 1st scene
          for speaker in metadata['segments'][scene1]['speakers']:
            if 'speakers' in metadata['segments'][scene2]:    # if speakers exist in 2nd scene
              if speaker not in metadata['segments'][scene2]['speakers']:
                num_edits += 1
            else:
                num_edits = len(metadata['segments'][scene1]['speakers'])  # all speakers have been deleted
        else:
          pass

        # 2. COUNTING ADDED SPEAKERS
        if 'speakers' in metadata['segments'][scene2]:
          for speaker in metadata['segments'][scene2]['speakers']:
            if 'speakers' in metadata['segments'][scene1]:
              if speaker not in metadata['segments'][scene1]['speakers']:
                num_edits += 1
            else:
                num_edits = len(metadata['segments'][scene2]['speakers']) # all speakers have been added
        else:
          pass

        #print("-------------Transition", scene1+1)
        #try:
        #  print("Speakers in Scene", scene1+1, metadata['segments'][scene1]['speakers'])
        #except:
        #  print("Speakers in Scene", scene1+1, 0)
        #try:
        #  print("Speakers in Scene", scene2+1, metadata['segments'][scene2]['speakers'])
        #except:
        #  print("Speakers in Scene", scene1+1, 0)
        #print("Number of edits (additions/deletions):", num_edits)

        if 'speakers' in metadata['segments'][scene1] and 'speakers' in metadata['segments'][scene2]:
            num_characters = len(list(set(metadata['segments'][scene1]['speakers'] + metadata['segments'][scene2]['speakers'])))
        else:
          try:
            num_characters = len(metadata['segments'][scene2]['speakers'])
          except:
            num_characters = len(metadata['segments'][scene1]['speakers'])

        #print("Number of speakers:", num_characters)
        segment_change_rate.append(num_edits/num_characters)
        #print("Segment change rate:", segment_change_rate)
        scene1 += 1
        scene2 += 1
        num_edits = 0
        num_characters = 0

      drama_change_rate = sum(segment_change_rate)/len(segment_change_rate)

  except:
    # deal with einakter
    drama_change_rate = 0

  return drama_change_rate

In [ ]:
# prepare empty columns

data = {'all_in_index': [],
        'final_scene_size': [],
        'drama_change_rate': []}

extra_features_2 = pd.DataFrame(data)
extra_features_2

,all_in_index,final_scene_size,drama_change_rate


In [ ]:
# test
'''
url = "https://dracor.org/api/corpora/ger/play/weise-masaniello"
print(url)
response = requests.get(url)
metadata = json.loads(response.text)
drama_change_rate = compute_change_rate(metadata)
print(drama_change_rate)
'''

'\nurl = "https://dracor.org/api/corpora/ger/play/weise-masaniello"\nprint(url)\nresponse = requests.get(url)\nmetadata = json.loads(response.text)\ndrama_change_rate = compute_change_rate(metadata)\nprint(drama_change_rate)\n'

In [ ]:
new_rows = []
for slug in tqdm(metadata_table['name']):
  url = f"http://dracor.org/api/v1/corpora/{language}/plays/{slug}"
  print("Computing Trilcke et al. 2017 metrics for", url)
  response = urlopen(url)
  metadata = json.loads(response.read())
  try:
    drama_change_rate = compute_change_rate(metadata)
    final_scene_size = compute_final_scene_size(metadata)
    all_in_index = compute_all_in_index(metadata)
  except Exception as e:
    print("This play throws an error:", url, e)
    continue

  new_row = {'all_in_index': all_in_index,
            'final_scene_size': final_scene_size,
            'drama_change_rate': drama_change_rate}

  new_rows.append(new_row)

# Create a DataFrame from the list of new rows
  extra_features_2 = pd.DataFrame(new_rows)

# Ensure indices are aligned
  extra_features_2.reset_index(drop=True, inplace=True)

  0%|          | 1/429 [00:00<00:59,  7.22it/s]

Computing Trilcke et al. 2017 metrics for http://dracor.org/api/v1/corpora/eng/plays/alexander-caesar
Computing Trilcke et al. 2017 metrics for http://dracor.org/api/v1/corpora/eng/plays/alexander-croesus


  1%|          | 3/429 [00:00<00:56,  7.57it/s]

Computing Trilcke et al. 2017 metrics for http://dracor.org/api/v1/corpora/eng/plays/alexander-darius
Computing Trilcke et al. 2017 metrics for http://dracor.org/api/v1/corpora/eng/plays/alexander-the-alexandraean-tragedy


  1%|          | 5/429 [00:00<00:57,  7.33it/s]

...

 99%|█████████▉| 426/429 [01:03<00:00,  6.80it/s]

Computing Trilcke et al. 2017 metrics for http://dracor.org/api/v1/corpora/eng/plays/wilson-the-three-lords-and-three-ladies-of-london
Computing Trilcke et al. 2017 metrics for http://dracor.org/api/v1/corpora/eng/plays/woodes-the-conflict-of-conscience


100%|█████████▉| 428/429 [01:04<00:00,  6.63it/s]

Computing Trilcke et al. 2017 metrics for http://dracor.org/api/v1/corpora/eng/plays/yarington-two-lamentable-tragedies
Computing Trilcke et al. 2017 metrics for http://dracor.org/api/v1/corpora/eng/plays/zouche-the-sophister


100%|██████████| 429/429 [01:04<00:00,  6.67it/s]


In [ ]:
'''
# For debugging purposes. Known errors:
# DIVISION BY ZERO --> the play has no scenes, only acts (segments = scenes)
# SPEAKERS --> a scene has no speakers, error in encoding or dumb scene
for url in errors:
  response = requests.get(url)
  data_json = json.loads(response.text)
  try:
    drama_change_rate = compute_change_rate(data_json)
  except Exception as e:
    print(url, e)
'''

'\n# For debugging purposes. Known errors:\n# DIVISION BY ZERO --> the play has no scenes, only acts (segments = scenes)\n# SPEAKERS --> a scene has no speakers, error in encoding or dumb scene\nfor url in errors:\n  response = requests.get(url)\n  data_json = json.loads(response.text)\n  try:\n    drama_change_rate = compute_change_rate(data_json)\n  except Exception as e:\n    print(url, e)\n'

In [ ]:
extra_features_2

,all_in_index,final_scene_size,drama_change_rate
0,1.0,0.272727,0.884226
1,1.0,0.272727,0.725000
2,1.0,0.214286,0.929167
3,1.0,0.437500,1.000000
4,1.0,1.000000,0.000000
...,...,...,...
420,1.0,1.000000,0.000000
421,1.0,0.966667,1.000000
422,1.0,0.058824,0.769048
423,1.0,1.000000,0.000000


In [ ]:
# add extra metrics to the table
table2 = pd.concat([table1, extra_features_2],axis=1)

In [ ]:
table2

,name,id,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,wordCountSp,wordCountStage,numOfP,numOfL,avg_betweenness,avg_closeness,avg_eigenvector,all_in_index,final_scene_size,drama_change_rate
0,alexander-caesar,ep000318,Tragedy,1604,11,0.478788,0.381818,1.600000,3.818182,3,...,21407,48,33,2584,0.054545,0.524548,0.266490,1.0,0.272727,0.884226
1,alexander-croesus,ep000315,Tragedy,1604,11,0.701948,0.327273,1.513514,3.272727,2,...,20156,28,10,2422,0.038384,0.459898,0.251954,1.0,0.272727,0.725000
2,alexander-darius,ep000316,Tragedy,1603,14,0.733333,0.274725,2.428571,3.571429,5,...,14453,33,45,1683,0.119048,0.424969,0.229104,1.0,0.214286,0.929167
3,alexander-the-alexandraean-tragedy,ep000317,Tragedy,1605,16,0.645238,0.291667,1.772152,4.375000,4,...,23337,36,23,2817,0.036310,0.390252,0.198640,1.0,0.437500,1.000000
4,anon-a-knack-to-know-a-knave,ep000117,Comedy,1592,29,1.000000,1.000000,1.000000,28.000000,1,...,16874,247,71,1654,0.000000,1.000000,0.185695,1.0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,wilson-the-three-ladies-of-london,ep000311,Comedy,1581,23,1.000000,1.000000,1.000000,22.000000,1,...,18765,512,1,1633,0.000000,1.000000,0.208514,1.0,0.545455,0.820355
425,wilson-the-three-lords-and-three-ladies-of-london,ep000310,0,1588,30,0.966667,0.933333,1.000000,27.066667,1,...,21390,1076,628,1121,0.000000,0.933333,0.179506,NaN,NaN,NaN
426,woodes-the-conflict-of-conscience,ep000799,Comedy,1572,17,0.695755,0.264706,1.733333,4.235294,3,...,19186,60,0,1946,0.037745,0.458304,0.207882,NaN,NaN,NaN
427,yarington-two-lamentable-tragedies,ep000313,Tragedy,1594,41,1.000000,1.000000,1.000000,40.000000,1,...,21421,765,8,2598,0.000000,1.000000,0.156174,NaN,NaN,NaN


In [ ]:
#final_table.to_csv(f"{language.capitalize()}DraCor_enriched_metadata.csv")

In [ ]:
#final_table = pd.read_csv('FreDraCor_enriched_metadata.csv')

#### 2. Adding metrics from Szemes and Vida 2022

In [ ]:
'''Average Character Per Scene: First, the average number of characters in one scene is calculated. To make texts of different
lengths comparable, this number is divided by the number of scenes.'''

def compute_averageCharinScene(metadata):
    from statistics import mean
    avg_char_scenes = []
    if len(metadata['segments']) > 1:
        if 'segments' in metadata:
            for segment in metadata['segments']:
                #print(len(segment['speakers']))
                if 'speakers' in segment:
                    avg_char_scenes.append(len(segment['speakers']))
                else:
                    avg_char_scenes.append(0)
    else: #deals with einakter
        avg_char_scenes.append(len(metadata['characters']))
    #print(avg_char_scenes)
    averageCharinScene = mean(avg_char_scenes)
    #averageCharinScene = mean / len(metadata['segments'])      # why dividing by the number of scenes? it's already an average
    averageCharinScene = round(averageCharinScene)

    return averageCharinScene

In [ ]:
#compute_averageCharinScene(metadata)

In [ ]:
'''Average Degree – Max Degree Ratio: Degree is the number of nodes that a node is connected to via edges. From this,
the average and maximum degree for the network as a whole can be calculated. This is done so that statements could be made
that pertain to the network itself. This measure therefore gives the proportion of the average degree number relative to the
maximum degree number, and in a sense resembles density – to what extent this measure can be considered an independent
characteristic that describes networks from a unique point of view will be discussed later.'''

def compute_avg_max_degree_ratio(network_metrics):
  try:
      avg_max_degree_ratio = network_metrics['averageDegree'] / network_metrics['maxDegree']
  except:
      avg_max_degree_ratio = 0
  return avg_max_degree_ratio

In [ ]:
#compute_avg_max_degree_ratio(network_metrics)

In [ ]:
'''Maximum Degree – Number Character Ratio: To get the total number of characters correct, one has to be subtracted
from the actual number of characters, since a node does not interact with itself. The ratio shows what fraction of the entire
cast does the node with the highest degree interact with.'''

def compute_max_degree_num_char_ratio(network_metrics):
  num_chars = len(network_metrics['nodes'])-1
  try:
      max_degree_num_char_ratio = network_metrics['maxDegree'] / num_chars
  except:
      max_degree_num_char_ratio = 0

  return max_degree_num_char_ratio

In [ ]:
#compute_max_degree_num_char_ratio(network_metrics)

In [ ]:
'''High Weigthed Degree, Medium Weigthed Degree, Low Weigthed Degree: The same method was employed to calculate the weighted degree
of characters as nodes. Here, the proportion of characters with many, medium or few connections in the network was
examined. Since the result is always adjusted to the number of characters, this measure is also size-independent, which allows us
to compare works with different numbers of characters. Our previous measures showed that the metrics of weighted degree (which
takes into account not only the number of edges leading into a node but also their “weight”, in this case the frequency of
appearing together in the same scene) better captures the differences between genres and the specific characteristics of plays
than using only the degree for the calculations.'''

def compute_weighted_degree_distribution(network_metrics):

    degrees = []
    for el in network_metrics['nodes']:
      degrees.append(el['weightedDegree'])
    degrees.sort()
    #print(degrees)

    low_degree = []
    medium_degree = []
    high_degree = []
    for el in network_metrics['nodes']:
      wgh_degree = el['weightedDegree']
      if wgh_degree < np.percentile(degrees, 33):      # using percentiles instead of kmeans, it's still size-independent
        low_degree.append(wgh_degree)
      elif wgh_degree < np.percentile(degrees, 66):
        medium_degree.append(wgh_degree)
      else:
        high_degree.append(wgh_degree)

    # compute distribution

    total_characters = len(network_metrics['nodes'])
    share_low = len(low_degree) / total_characters
    share_medium = len(medium_degree) / total_characters
    share_high = len(high_degree) / total_characters

    return share_low, share_medium, share_high

In [ ]:
#compute_weighted_degree_distribution(network_metrics)

In [ ]:
'''High Speech, Medium Speech, Low Speech: [...] k-means clustering was employed (k=3, likewise) to distinguish between characters
in plays who are not just members of a group in the stage (for which DraCor uses the annotation “isGroup = True”, e.g. “street vendors”).
The three groups were then ranked along their mean value, and the number of characters in each category was then adjusted to the
total number of non-group characters in the play. This ratio makes it possible to compare plays with different numbers of characters,
since it shows not only the number of characters who speak a lot/medium/less in a text, but also the proportion of characters who
do so, which essentially gives us an idea of the distribution of linguistic complexity among characters. '''

def compute_speech_intensity(spoken_text):
    speech_length = []
    for character in spoken_text:
      char_speech = len(re.findall(r'\w+', str(character['text'])))
      speech_length.append(char_speech)
    speech_length.sort()
    #print("SL", speech_length)

    lowSpeech = 0
    mediumSpeech = 0
    highSpeech = 0
    from statistics import mean
    for character in spoken_text:
      char_speech = len(re.findall(r'\w+', str(character['text'])))
      #if character['isGroup'] == False:         # why excluding them? they might be reintegrated
      #print(character['id'], len(re.findall(r'\w+', str(character['text']))))
      if char_speech < np.percentile(speech_length, 33):      # using percentiles instead of kmeans, it's still size-independent
        lowSpeech += 1
      elif char_speech < np.percentile(speech_length, 66):
        mediumSpeech += 1
      else:
        highSpeech += 1

    # compute distribution

    total_characters = len(spoken_text)
    share_low = lowSpeech / total_characters
    share_medium = mediumSpeech / total_characters
    share_high = highSpeech / total_characters

    return share_low, share_medium, share_high

In [ ]:
#compute_speech_intensity(spoken_text)

In [ ]:
'''Average Character Speech: This shows the average number of words spoken by a character
in a play. In Szemes/Vidas implementation, only characters who speak more than 10 times were considered,
but we decided to consider all of them.'''

def compute_averageCharSpeech(character_metrics):
    avg_list = []
    for character in character_metrics:
        #if character["numOfSpeechActs"] < 10:  # Skip characters with less than 10 lines
        #    continue
        if character["numOfSpeechActs"] != 0:
          avg_char = character["numOfWords"]/character["numOfSpeechActs"]
        else:
          character["numOfSpeechActs"] = 1
          avg_char = character["numOfWords"]/character["numOfSpeechActs"]
        print(character["id"], avg_char)
        avg_list.append(avg_char)
    averageCharSpeech = mean(avg_list)

    return averageCharSpeech

In [ ]:
'''
characters_url = "http://dracor.org/api/v1/corpora/fre/plays/racine-phedre/characters"
response3 = requests.get(characters_url)
character_metrics = response3.json()
'''

In [ ]:
def load_all_metadata(slug, language):
    metadata_url = f"http://dracor.org/api/v1/corpora/{language}/plays/{slug}"
    response1 = requests.get(metadata_url)
    metadata = response1.json() if response1.status_code == 200 else print("Error: cannot load metadata")

    spoken_text_url = f"http://dracor.org/api/v1/corpora/{language}/plays/{slug}/spoken-text-by-character"
    response2 = requests.get(spoken_text_url)
    spoken_text = response2.json() if response2.status_code == 200 else print("Error: cannot load spoken text")

    network_metrics_url = f"http://dracor.org/api/v1/corpora/{language}/plays/{slug}/metrics"
    response3 = requests.get(network_metrics_url)
    network_metrics = response3.json() if response3.status_code == 200 else print("Error: cannot load network metrics")

    characters_url = f"http://dracor.org/api/v1/corpora/{language}/plays/{slug}/characters"
    response4 = requests.get(characters_url)
    character_metrics = response4.json() if response4.status_code == 200 else print("Error: cannot load character metrics")

    return metadata, spoken_text, network_metrics, character_metrics

In [ ]:
# prepare empty columns

data = {'averageCharinScene': [],
        'avg_max_degree_ratio': [],
        'max_degree_num_char_ratio': [],
        'lowWeightedDegree': [],
        'mediumWeightedDegree': [],
        'highWeightedDegree': [],
        'averageCharSpeech':[],
        'lowSpeech': [],
        'mediumSpeech': [],
        'highSpeech': []}

extra_features_3 = pd.DataFrame(data)
extra_features_3

,averageCharinScene,avg_max_degree_ratio,max_degree_num_char_ratio,lowWeightedDegree,mediumWeightedDegree,highWeightedDegree,averageCharSpeech,lowSpeech,mediumSpeech,highSpeech


In [ ]:
# debug
"""
slug = "hausted-the-rival-friends"
response = load_all_metadata(slug,"eng")
metadata = response[0]
spoken_text = response[1]
network_metrics = response[2]
characters_metrics = response[3]

averageCharinScene = compute_averageCharinScene(metadata)
avg_max_degree_ratio = compute_avg_max_degree_ratio(network_metrics)
max_degree_num_char_ratio = compute_max_degree_num_char_ratio(network_metrics)
lowWeightedDegree, mediumWeightedDegree, highWeightedDegree = compute_weighted_degree_distribution(network_metrics)
lowSpeech, mediumSpeech, highSpeech = compute_speech_intensity(spoken_text)
averageCharSpeech = compute_averageCharSpeech(character_metrics)
"""

In [ ]:
language = language
for slug in tqdm(metadata_table['name'].tolist()):
  print(f"------> Loading {slug}...")
  try:
      response = load_all_metadata(slug, language)
      metadata = response[0]
      spoken_text = response[1]
      network_metrics = response[2]
      character_metrics = response [3]
  except Exception as e:
      print(slug, "throws an error:" ,e)

  averageCharinScene = compute_averageCharinScene(metadata)
  avg_max_degree_ratio = compute_avg_max_degree_ratio(network_metrics)
  max_degree_num_char_ratio = compute_max_degree_num_char_ratio(network_metrics)
  lowWeightedDegree, mediumWeightedDegree, highWeightedDegree = compute_weighted_degree_distribution(network_metrics)
  try:
    lowSpeech, mediumSpeech, highSpeech = compute_speech_intensity(spoken_text)
  except Exception as e:
    print("Setting values at 0 for", slug)
    lowSpeech, mediumSpeech, highSpeech = 0, 0, 0 # deal with errors, is this suitable?
  averageCharSpeech = compute_averageCharSpeech(character_metrics)

#  except Exception as e:
 #   print("This play throws an error:", url, e)

    #insert all other functions

  print('Computed all Szemes/Vida metrics for', slug)

  new_row = {'averageCharinScene': averageCharinScene,
        'avg_max_degree_ratio': avg_max_degree_ratio,
        'max_degree_num_char_ratio': max_degree_num_char_ratio,
        'lowWeightedDegree': lowWeightedDegree,
        'mediumWeightedDegree': mediumWeightedDegree,
        'highWeightedDegree': highWeightedDegree,
        'averageCharSpeech':averageCharSpeech,
        'lowSpeech': lowSpeech,
        'mediumSpeech': mediumSpeech,
        'highSpeech': highSpeech}


  extra_features_3 = pd.concat([extra_features_3, pd.DataFrame([new_row])], ignore_index=True)



  0%|          | 0/429 [00:00<?, ?it/s]

------> Loading alexander-caesar...


  0%|          | 1/429 [00:03<25:54,  3.63s/it]

Computed all Szemes/Vida metrics for alexander-caesar
------> Loading alexander-croesus...


  0%|          | 2/429 [00:07<24:51,  3.49s/it]

...

Computed all Szemes/Vida metrics for woodes-the-conflict-of-conscience
------> Loading yarington-two-lamentable-tragedies...


100%|█████████▉| 428/429 [25:45<00:03,  3.71s/it]

Computed all Szemes/Vida metrics for yarington-two-lamentable-tragedies
------> Loading zouche-the-sophister...


100%|██████████| 429/429 [25:48<00:00,  3.61s/it]

Computed all Szemes/Vida metrics for zouche-the-sophister


In [ ]:
extra_features_3

,averageCharinScene,avg_max_degree_ratio,max_degree_num_char_ratio,lowWeightedDegree,mediumWeightedDegree,highWeightedDegree,averageCharSpeech,lowSpeech,mediumSpeech,highSpeech
0,3.0,0.636364,0.600000,0.363636,0.090909,0.545455,303.580529,0.363636,0.272727,0.363636
1,3.0,0.409091,0.800000,0.363636,0.272727,0.363636,217.072845,0.363636,0.272727,0.363636
2,3.0,0.595238,0.461538,0.357143,0.071429,0.571429,96.536216,0.357143,0.285714,0.357143
3,3.0,0.437500,0.666667,0.312500,0.187500,0.500000,294.300796,0.312500,0.312500,0.375000
4,29.0,1.000000,1.000000,0.000000,0.000000,1.000000,44.234548,0.344828,0.310345,0.344828
...,...,...,...,...,...,...,...,...,...,...
424,23.0,1.000000,1.000000,0.000000,0.000000,1.000000,39.951322,0.347826,0.304348,0.347826
425,15.0,0.966667,0.965517,0.033333,0.000000,0.966667,30.544488,0.333333,0.333333,0.333333
426,3.0,0.325792,0.812500,0.294118,0.294118,0.411765,118.328079,0.352941,0.294118,0.352941
427,41.0,1.000000,1.000000,0.000000,0.000000,1.000000,33.629712,0.341463,0.317073,0.341463


In [ ]:
extra_features_3.reset_index(drop=True, inplace=True)
table3 = pd.concat([table2, extra_features_3],axis=1)

In [ ]:
table3

,name,id,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,averageCharinScene,avg_max_degree_ratio,max_degree_num_char_ratio,lowWeightedDegree,mediumWeightedDegree,highWeightedDegree,averageCharSpeech,lowSpeech,mediumSpeech,highSpeech
0,alexander-caesar,ep000318,Tragedy,1604,11,0.478788,0.381818,1.600000,3.818182,3,...,3.0,0.636364,0.600000,0.363636,0.090909,0.545455,303.580529,0.363636,0.272727,0.363636
1,alexander-croesus,ep000315,Tragedy,1604,11,0.701948,0.327273,1.513514,3.272727,2,...,3.0,0.409091,0.800000,0.363636,0.272727,0.363636,217.072845,0.363636,0.272727,0.363636
2,alexander-darius,ep000316,Tragedy,1603,14,0.733333,0.274725,2.428571,3.571429,5,...,3.0,0.595238,0.461538,0.357143,0.071429,0.571429,96.536216,0.357143,0.285714,0.357143
3,alexander-the-alexandraean-tragedy,ep000317,Tragedy,1605,16,0.645238,0.291667,1.772152,4.375000,4,...,3.0,0.437500,0.666667,0.312500,0.187500,0.500000,294.300796,0.312500,0.312500,0.375000
4,anon-a-knack-to-know-a-knave,ep000117,Comedy,1592,29,1.000000,1.000000,1.000000,28.000000,1,...,29.0,1.000000,1.000000,0.000000,0.000000,1.000000,44.234548,0.344828,0.310345,0.344828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,wilson-the-three-ladies-of-london,ep000311,Comedy,1581,23,1.000000,1.000000,1.000000,22.000000,1,...,23.0,1.000000,1.000000,0.000000,0.000000,1.000000,39.951322,0.347826,0.304348,0.347826
425,wilson-the-three-lords-and-three-ladies-of-london,ep000310,0,1588,30,0.966667,0.933333,1.000000,27.066667,1,...,15.0,0.966667,0.965517,0.033333,0.000000,0.966667,30.544488,0.333333,0.333333,0.333333
426,woodes-the-conflict-of-conscience,ep000799,Comedy,1572,17,0.695755,0.264706,1.733333,4.235294,3,...,3.0,0.325792,0.812500,0.294118,0.294118,0.411765,118.328079,0.352941,0.294118,0.352941
427,yarington-two-lamentable-tragedies,ep000313,Tragedy,1594,41,1.000000,1.000000,1.000000,40.000000,1,...,41.0,1.000000,1.000000,0.000000,0.000000,1.000000,33.629712,0.341463,0.317073,0.341463


In [ ]:
#df = final_table.loc[:,~final_table.columns.duplicated()].copy()

In [ ]:
#df.dropna(how='all')

#### 3. Adding metrics from Algee-Hewitt 2017

In [ ]:
# calculating the distribution of eigenvector centrality

# Gini coefficient
# modified from https://planspace.org/2013/06/21/how-to-calculate-gini-coefficient-from-raw-data-in-python/
def gini(network_metrics):
    eigenvectors = []
    for char in network_metrics['nodes']:
      eigenvectors.append(char['eigenvector']) # creates eigenvectors list
    #print(eigenvectors)
    sorted_list = sorted(eigenvectors)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(eigenvectors) / 2.
    gini_coefficient = (fair_area - area) / fair_area

    return gini_coefficient

In [ ]:
'''
For a single metric based on EC, then, we can combine these two measurements, scaling the Gini coefficient
by dividing it by the percentage of characters in the top quartile of the eigenvector distribution and
creating a ratio between them. [...] Moreover, because both of these metrics offer information on the
function of the protagonist (whether the strength of the core/periphery divide, or the distribution
of protagonists within the core), their combination in this ratio yields a measurement that I call
“protagonism.”
'''
def compute_protagonism(network_metrics):
    counter_chars = 0
    counter_top_quartile = 0
    eigenvectors = []
    from statistics import mean
    for char in network_metrics['nodes']:
      counter_chars += 1
      eigenvectors.append(char['eigenvector']) # creates eigenvectors list
      egv = char['eigenvector']
      if egv > np.percentile(eigenvectors, 75):      # top quartile
          counter_top_quartile +=1
    #print("Characters", counter_chars)
    #print("Characters in top 25%", counter_top_quartile)
    gini_coefficient = gini(network_metrics)
    percent_top_25 = (counter_top_quartile*100) / counter_chars
    #print("Gini", gini_coefficient)
    #print('Percent', percent_top_25)
    try:
        protagonism = gini_coefficient / percent_top_25
    except: # deals with e.g. fully-connected plays, where Gini = 0 and all egv are equal
        protagonism = 0

    return protagonism

In [ ]:
# test
'''
network_metrics_url = "http://dracor.org/api/corpora/{language}/play/barbieri-l-inavertito/metrics"
response2 = urlopen(network_metrics_url)
network_metrics = json.loads(response2.read())
compute_protagonism(network_metrics)
'''

In [ ]:
# mediateness = maximum normalized betweenness of a play’s characters

def compute_mediateness(network_metrics):
    betweenesses = []
    for char in network_metrics['nodes']:
      betweenesses.append(char['betweenness'])
    betweenesses = preprocessing.normalize([betweenesses])
    mediateness = betweenesses.max()

    return(mediateness)

# what is meant here by normalisation?

Results in my implementation are very different from Algee-Hewitt's... Why?

1) he didn't worked with the same (clean) DraCor data

2) it is not 100% clear how he calculated protagonism and mediateness -- mine is an approximation

In [ ]:
# prepare empty columns

data = {'protagonism': [],
        'mediateness': []}

extra_features_4 = pd.DataFrame(data)
extra_features_4

,protagonism,mediateness


In [ ]:
for slug in tqdm(metadata_table['name']):
  print(f"------> Loading {slug}...")
  try:
      response = load_all_metadata(slug, language)
      network_metrics = response[2]
  except Exception as e:
      print("This play throws an error:", slug, e)

  try:
      protagonism = compute_protagonism(network_metrics)
  except Exception as e:
      print("This play throws an error:", slug, e)
  try:
      mediateness = compute_mediateness(network_metrics)
  except Exception as e:
      print("This play throws an error:", slug, e)

  print('Computed all Algee-Hewitt metrics for', slug)

  new_row = {'protagonism': protagonism,
        'mediateness': mediateness}

  extra_features_4 = pd.concat([extra_features_4, pd.DataFrame([new_row])], ignore_index=True)

  0%|          | 0/429 [00:00<?, ?it/s]

------> Loading alexander-caesar...


  0%|          | 1/429 [00:03<26:35,  3.73s/it]

Computed all Algee-Hewitt metrics for alexander-caesar
------> Loading alexander-croesus...


  0%|          | 2/429 [00:07<24:51,  3.49s/it]

...

Computed all Algee-Hewitt metrics for woodes-the-conflict-of-conscience
------> Loading yarington-two-lamentable-tragedies...


100%|█████████▉| 428/429 [26:28<00:03,  3.75s/it]

Computed all Algee-Hewitt metrics for yarington-two-lamentable-tragedies
------> Loading zouche-the-sophister...


100%|██████████| 429/429 [26:31<00:00,  3.71s/it]

Computed all Algee-Hewitt metrics for zouche-the-sophister


In [ ]:
extra_features_4

In [ ]:
extra_features_4.reset_index(drop=True, inplace=True)

In [ ]:
final_table = pd.concat([table3, extra_features_4],axis=1)
final_table

,name,id,normalizedGenre,yearNormalized,size,averageClustering,density,averagePathLength,averageDegree,diameter,...,max_degree_num_char_ratio,lowWeightedDegree,mediumWeightedDegree,highWeightedDegree,averageCharSpeech,lowSpeech,mediumSpeech,highSpeech,protagonism,mediateness
0,alexander-caesar,ep000318,Tragedy,1604,11,0.478788,0.381818,1.600000,3.818182,3,...,0.600000,0.363636,0.090909,0.545455,303.580529,0.363636,0.272727,0.363636,0.010636,0.673300
1,alexander-croesus,ep000315,Tragedy,1604,11,0.701948,0.327273,1.513514,3.272727,2,...,0.800000,0.363636,0.272727,0.363636,217.072845,0.363636,0.272727,0.363636,0.040553,0.991325
2,alexander-darius,ep000316,Tragedy,1603,14,0.733333,0.274725,2.428571,3.571429,5,...,0.461538,0.357143,0.071429,0.571429,96.536216,0.357143,0.285714,0.357143,0.046904,0.666858
3,alexander-the-alexandraean-tragedy,ep000317,Tragedy,1605,16,0.645238,0.291667,1.772152,4.375000,4,...,0.666667,0.312500,0.187500,0.500000,294.300796,0.312500,0.312500,0.375000,0.033527,0.885140
4,anon-a-knack-to-know-a-knave,ep000117,Comedy,1592,29,1.000000,1.000000,1.000000,28.000000,1,...,1.000000,0.000000,0.000000,1.000000,44.234548,0.344828,0.310345,0.344828,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,wilson-the-three-ladies-of-london,ep000311,Comedy,1581,23,1.000000,1.000000,1.000000,22.000000,1,...,1.000000,0.000000,0.000000,1.000000,39.951322,0.347826,0.304348,0.347826,0.000000,0.000000
425,wilson-the-three-lords-and-three-ladies-of-london,ep000310,0,1588,30,0.966667,0.933333,1.000000,27.066667,1,...,0.965517,0.033333,0.000000,0.966667,30.544488,0.333333,0.333333,0.333333,0.000000,0.000000
426,woodes-the-conflict-of-conscience,ep000799,Comedy,1572,17,0.695755,0.264706,1.733333,4.235294,3,...,0.812500,0.294118,0.294118,0.411765,118.328079,0.352941,0.294118,0.352941,0.028274,0.989945
427,yarington-two-lamentable-tragedies,ep000313,Tragedy,1594,41,1.000000,1.000000,1.000000,40.000000,1,...,1.000000,0.000000,0.000000,1.000000,33.629712,0.341463,0.317073,0.341463,0.000000,0.000000


In [ ]:
final_table.plot.bar(x='name', y='size',figsize=(100,80))

<Axes: xlabel='name'>

In [ ]:
final_table.to_csv(f"{language.capitalize()}DraCor_enriched_metadata")

# Trashbin

In [ ]:
# check which rows have null values

# Read CSV from URL
url = 'https://raw.githubusercontent.com/lucagiovannini7/emdracor/main/metadata/EmDraCor_enriched_metadata.csv'
df = pd.read_csv(url)

# Find and print cells with null values
counter = 0
for index, row in df.iterrows():
    for col in df.columns:
        if row[col] == 0.0:
            counter += 1
            #print(f"Metric ({col}) in work ({row['name']}) is 0.0")
print((415/6600)*100)

6.287878787878788


In [ ]:
url = "http://dracor.org/api/corpora/{language}/play/schoch-comoedia-studenten"
response = requests.get(url)
data_json = json.loads(response.text)
drama_change_rate = compute_change_rate(data_json)
#print(data_json['segments'][3]['speakers'])

In [ ]:
'''Average Character Speech: This shows the average number of words spoken by a character
in a play. Only characters who speak more than 10 times were considered.'''

def compute_averageCharSpeech(spoken_text):
    all_avg_num_words = []

    for character in spoken_text:
        if len(character['text']) < 10:  # Skip characters with less than 10 lines
            continue

        char_avg_num_words = []
        for line in character['text']:
            char_avg_num_words.append(len(re.findall(r'\w+', line)))

        mean_char = mean(char_avg_num_words)
        mean_char = round(mean_char)
        print(f"Character: {character['label']}, Average words per line: {mean_char}")
        all_avg_num_words.append(mean_char)

    averageCharSpeech = mean(all_avg_num_words)
    averageCharSpeech = round(averageCharSpeech, 2)

    return averageCharSpeech